In [ ]:
%matplotlib notebook
from rfsoc_rfdc.rfsoc_overlay import RFSoCOverlay
from rfsoc_rfdc.overlay_task import OverlayTask
from rfsoc_rfdc.overlay_task import BlinkLedTask

from rfsoc_rfdc.transmitter.single_ch_transmitter_task import SingleChTransmitterTask
from rfsoc_rfdc.receiver.single_ch_receiver_task import SingleChReceiverTask

from rfsoc_rfdc.transmitter.multi_ch_transmitter_task import MultiChTransmitterTask
from rfsoc_rfdc.receiver.multi_ch_receiver_task import MultiChReceiverTask

from rfsoc_rfdc.rfdc_task import RfdcTask 
from rfsoc_rfdc.cmac_task import CmacTask

from rfsoc_rfdc.rfdc_config import ZCU216_CONFIG
import sys
import os
import time

In [ ]:
ol = RFSoCOverlay(path_to_bitstream="./rfsoc_rfdc/bitstream/rfsoc_rfdc_v32_4G8x.bit")

NEW_CONFIG = {
    "RefClockForPLL": 500.0,
    "DACSampleRate": 4000.0,
    "DACInterpolationRate": 8,
    "DACNCO": 0,
    "ADCSampleRate": 2000.0,
    "ADCInterpolationRate": 20,
    "ADCNCO": -0
}
ZCU216_CONFIG.update(NEW_CONFIG)

In [ ]:
from rfsoc_rfdc.rfdc import MyRFdc
from rfsoc_rfdc.clocks import set_custom_lmclks

print(f"Configure LMK and LMX chips.")
set_custom_lmclks()

print(f"Initialize RF Data Converter")
my_rfdc = MyRFdc(ol, debug_mode=False)
my_rfdc.init_setup()

In [ ]:
print(f"Start transmitter")

led_t = BlinkLedTask(ol)
led_t.start()

for amp in range(13+1):
    VAR = {"DAC_SCALING_FACTOR" : 2**(-amp)}
    print(f"Change DAC_SCALING_FACTOR to 2**-{amp}")
    ZCU216_CONFIG.update(VAR)

    tx_t = SingleChTransmitterTask(ol, "./wave_files/wifi_wave.mat")

    for tile in my_rfdc.dac_tiles:
        my_rfdc.config_dac_blocks(tile)
    
    tx_t.start()
    time.sleep(20)
    tx_t.stop()
    
print(f"End of tests")
led_t.stop()